In [ ]:
import MDAnalysis as mda
from MDAnalysis.lib.distances import capped_distance
import numpy as np
import matplotlib.pyplot as plt

systems = {
    "300": ("./Neuropeptide1FVN_Urea/1FVN_80MU/npt.gro", "./Neuropeptide1FVN_Urea/1FVN_80MU/md_cntr.xtc"),
    "325": ("./Neuropeptide1FVN325_Urea/1FVN_80MU/npt.gro", "./Neuropeptide1FVN325_Urea/1FVN_80MU/md_cntr.xtc"),
    "350": ("./Neuropeptide1FVN350_Urea/1FVN_80MU/npt.gro", "./Neuropeptide1FVN350_Urea/1FVN_80MU/md_cntr.xtc"),
    "375": ("./Neuropeptide1FVN375_Urea/1FVN_80MU/npt.gro", "./Neuropeptide1FVN375_Urea/1FVN_80MU/md_cntr.xtc"),
}

peptide_sel = "resid 15-30 and name CA"   # peptide region under consideration
urea_sel    = "resname Urea and name H*"  # check urea name in topology
cutoff      = 5.0             # Å
n_atoms_per_urea = 8          # divide by this if you want molecules not atoms

#function
def compute_avg_counts(top, traj, peptide_sel, urea_sel, cutoff, n_atoms_per_urea):
    u = mda.Universe(top, traj)
    peptide = u.select_atoms(peptide_sel)
    urea = u.select_atoms(urea_sel)
    resids = [res.resid for res in peptide.residues]
    counts = np.zeros((len(u.trajectory), len(resids)), dtype=int)

    for ts in u.trajectory:
        for i, res in enumerate(peptide.residues):
            dists = capped_distance(res.atoms.positions,
                                    urea.positions,
                                    max_cutoff=cutoff,
                                    box=u.dimensions)[0]
            counts[ts.frame, i] = len(dists)

    avg_counts = counts.mean(axis=0) / n_atoms_per_urea
    return resids, avg_counts

#Collect all concentrations
results = {}
for label, (top, traj) in systems.items():
    resids, avg_counts = compute_avg_counts(top, traj, peptide_sel, urea_sel, cutoff, n_atoms_per_urea)
    results[label] = avg_counts

#Plot as lines
plt.figure(figsize=(10,6))
for label, avg_counts in results.items():
    plt.plot(resids, avg_counts, marker='o', label=label)

plt.xlabel("Residue ID")
plt.ylabel(f"Avg. # Urea molecules")
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import rdf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


systems = {
    "0.3": ("./Neuropeptide1FVN375_Urea/1FVN_20MU/npt.gro", "./Neuropeptide1FVN375_Urea/1FVN_20MU/md_500.xtc"),
    "0.6": ("./Neuropeptide1FVN375_Urea/1FVN_40MU/npt.gro", "./Neuropeptide1FVN375_Urea/1FVN_40MU/md_500.xtc"),
    "0.9": ("./Neuropeptide1FVN375_Urea/1FVN_60MU/npt.gro", "./Neuropeptide1FVN375_Urea/1FVN_60MU/md_500.xtc"),
    "1.2": ("./Neuropeptide1FVN375_Urea/1FVN_80MU/npt.gro", "./Neuropeptide1FVN375_Urea/1FVN_80MU/md_500.xtc"),
}

#  Parameters 
peptide_sel = "resid 15-30 and name CA"
urea_sel = "resname Urea"
water_sel = "resname SOL and name OW"
cutoff = 5.0  # Å
nbins = 100


results = {}  # {conc: {resid: (Gurea_corr, Gwater_corr, Gamma)}}

for conc, (top_file, traj_file) in systems.items():
    print(f"\nProcessing concentration: {conc} M")

    u = mda.Universe(top_file, traj_file)

    peptide = u.select_atoms(peptide_sel)
    urea = u.select_atoms(urea_sel)
    water = u.select_atoms(water_sel)

    # Bulk densities
    box_vol = np.mean([ts.volume for ts in u.trajectory])  # Å^3
    rho_urea = len(urea) / box_vol
    rho_water = len(water) / box_vol
    print(f"ρ(urea)={rho_urea:.4e}, ρ(water)={rho_water:.4e}")
    print(f"Box volume (Å^3): {box_vol:.3e}")
    conc_data = {}

    for res in peptide.residues:
        res_sel = res.atoms

        # RDF residue–urea
        rdf_ur = rdf.InterRDF(res_sel, urea, range=(0.0, cutoff), nbins=nbins)
        rdf_ur.run()
        r = rdf_ur.bins
        g_r_ur = rdf_ur.rdf

        # RDF residue–water
        rdf_w = rdf.InterRDF(res_sel, water, range=(0.0, cutoff), nbins=nbins)
        rdf_w.run()
        g_r_w = rdf_w.rdf

        # Truncated KB integrals 
        dr = r[1] - r[0]
        G_ur = 4 * np.pi * np.sum((g_r_ur - 1.0) * r**2 * dr)
        G_w  = 4 * np.pi * np.sum((g_r_w  - 1.0) * r**2 * dr)

        #  Krüger correction 
        V_R = 4/3 * np.pi * cutoff**3

        N_in_ur = 4 * np.pi * rho_urea * np.sum(g_r_ur * r**2 * dr)
        N_in_w  = 4 * np.pi * rho_water * np.sum(g_r_w  * r**2 * dr)

        rho_in_ur = N_in_ur / V_R
        rho_in_w  = N_in_w  / V_R

        C_ur = V_R * (rho_in_ur - rho_urea)
        C_w  = V_R * (rho_in_w  - rho_water)

        G_ur_corr = G_ur - C_ur
        G_w_corr  = G_w  - C_w

        # Preferential binding coefficient
        gamma_i = rho_urea * G_ur_corr - rho_water * G_w_corr

        conc_data[res.resid] = (G_ur_corr, G_w_corr, gamma_i)

    results[conc] = conc_data

# Convert to matrices
resids = sorted(list(results[list(systems.keys())[0]].keys()))
Gamma_matrix = np.zeros((len(systems), len(resids)))
Gurea_matrix = np.zeros_like(Gamma_matrix)
Gwater_matrix = np.zeros_like(Gamma_matrix)

for ci, conc in enumerate(systems.keys()):
    for ri, resid in enumerate(resids):
        G_ur, G_w, gamma = results[conc][resid]
        Gurea_matrix[ci, ri] = G_ur
        Gwater_matrix[ci, ri] = G_w
        Gamma_matrix[ci, ri] = gamma

        #  Heatmap of Γ 
plt.figure(figsize=(10,6))
vmin = np.min(Gamma_matrix)
vmax = np.max(Gamma_matrix)

# Symmetric limits around 0 for diverging colormap
abs_max = 13
abs_min = 5
im = plt.imshow(
    Gamma_matrix,
    aspect="auto",
    cmap="rainbow",
    extent=[resids[0]-0.5, resids[-1]+0.5, -0.5, len(systems)-0.5],
    norm=mpl.colors.Normalize(vmin=abs_min, vmax=abs_max)
)
plt.colorbar(im, label=r"$\Gamma_i$")

plt.yticks(range(len(systems)), list(systems.keys()))
plt.xlabel("Residue ID")
plt.ylabel("Concentration (M)")
plt.tight_layout()
plt.savefig("heatmap375_500.png", dpi=600, bbox_inches="tight")
plt.show()

# Print Gurea, Gwater, and Γ values 
print("\nResidue-wise Gurea, Gwater, and Γ values:")
for conc in systems.keys():
    print(f"\nConcentration {conc} M")
    for resid in resids:
        G_ur, G_w, gamma = results[conc][resid]
        print(f"Residue {resid:3d}:  Gurea = {G_ur: .6f},  Gwater = {G_w: .6f},  Γ = {gamma: .6f}")